# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell, Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

    CondaPkg Found dependencies: /home/runner/.julia/packages/ASEconvert/CNQ1A/CondaPkg.toml
    CondaPkg Found dependencies: /home/runner/.julia/packages/PythonCall/dsECZ/CondaPkg.toml
    CondaPkg Dependencies already up to date


As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123


Python None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298544780507                   -0.85    5.0         
  2   -8.300203209304       -2.78       -1.25    1.0    1.12s
  3   -8.300434826392       -3.64       -1.89    3.1    292ms
  4   -8.300461688771       -4.57       -2.78    1.6    179ms
  5   -8.300464366728       -5.57       -3.11    3.1    225ms
┌ Warning: Eigensolver not converged
│   iterations =
│    8-element Vector{Int64}:
│     100
│       1
│       2
│       2
│       1
│       2
│       1
│       1
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/scf/self_consistent_field.jl:35
  6   -8.300464517979       -6.82       -3.30   13.8    1.28s
  7   -8.300464579744       -7.21       -3.45    2.5    212ms
  8   -8.300464613741       -7.47       -3.60    1.1   

In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.67448891654                   -0.70    6.0         
  2   -16.67864093675       -2.38       -1.14    1.0    449ms
  3   -16.67920827087       -3.25       -1.87    3.8    554ms
  4   -16.67927460841       -4.18       -2.77    3.2    493ms
  5   -16.67928588593       -4.95       -3.19    4.5    631ms
  6   -16.67928619233       -6.51       -3.48    2.5    495ms
  7   -16.67928621813       -7.59       -3.96    1.8    426ms
  8   -16.67928622142       -8.48       -4.56    2.8    481ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32377916221                   -0.56    6.4         
  2   -33.33249027036       -2.06       -1.00    1.8    1.55s
  3   -33.33403484454       -2.81       -1.74    3.9    1.85s
  4   -33.33505730959       -2.99       -2.29    3.5    1.97s
  5   -33.33689969642       -2.73       -2.50   10.8    2.81s
  6   -33.33693309005       -4.48       -2.79    4.5    1.80s
  7   -33.33694359370       -4.98       -3.69    2.4    1.85s
  8   -33.33694369589       -6.99       -3.88    4.8    2.27s
  9   -33.33694377682       -7.09       -4.40    2.0    1.47s


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298461830450                   -0.85    5.0         
  2   -8.300261436214       -2.74       -1.59    1.0    122ms
  3   -8.300425411143       -3.79       -2.56    2.9    163ms
  4   -8.300377721693   +   -4.32       -2.28    4.6    258ms
  5   -8.300464041968       -4.06       -3.36    1.0    174ms
  6   -8.300464574090       -6.27       -3.74    3.8    241ms
  7   -8.300464640646       -7.18       -4.33    2.0    175ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32535772659                   -0.56    7.6         
  2   -33.30232389117   +   -1.64       -1.26    1.0    1.26s
  3   -3.182453190301   +    1.48       -0.33    6.6    3.86s
  4   -33.30047104558        1.48       -1.46    6.1    2.74s
  5   -33.29166480617   +   -2.06       -1.45    2.9    1.91s
  6   -32.88470619995   +   -0.39       -1.24    3.2    1.84s
  7   -33.25986237454       -0.43       -1.61    3.6    1.87s
  8   -33.33652553566       -1.12       -2.39    3.0    1.65s
  9   -33.33672878379       -3.69       -2.63    3.9    2.16s
 10   -33.33668436894   +   -4.35       -2.59    2.4    1.75s
 11   -33.33690321084       -3.66       -3.04    1.8    1.53s
 12   -33.33693864308       -4.45      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.